# Douyin Live Recorder

Notebook นี้ใช้สำหรับรัน DouyinLiveRecorder เพื่อบันทึกวิดีโอไลฟ์จาก Douyin (TikTok จีน)

In [ ]:
# Clone repositories
!git clone https://github.com/ihmily/DouyinLiveRecorder.git
!git clone https://github.com/chaiz64/l777k.git

In [ ]:
# Copy configuration files
!cp config.ini DouyinLiveRecorder/config/
!cp URL_config.ini DouyinLiveRecorder/config/

In [ ]:
# เข้าไปยังโฟลเดอร์โปรเจค
%cd DouyinLiveRecorder

# ติดตั้ง dependencies
!pip3 install -r requirements.txt

In [ ]:
# ติดตั้ง ffmpeg
!apt update
!apt install ffmpeg -y

In [ ]:
# รันโปรแกรม
!python main.py